`splatlog.setup` Notes
==============================================================================

`splatlog.setup` is the function you need to call to get any logging output,
which is typically what you're looking for when you're throwing something
together quickly or jamming `splatlog` in because you need some help seeing
what's going on.

To this end, the goal of `splatlog.setup` is to be "reference-free", in the
sense that you don't need to look anything up to get what you want out of it.
That means that, ideally, whatever makes sense to write in a `setup` call should
_just work_ if reasonably possible.

That aside, `setup` should be documented, and it should be documented well. At
the moment, that's not the case.

This notebook is dedicated to:

1.  Fiddling around when writing the `splatlog.setup` doc.
2.  Exploring and cataloging the ways it "makes sense" to me to call `setup`,
    to see if we can support them.

In [12]:
import splatlog
from collections.abc import Mapping
from typing import Any
from rich.console import Console
import rich.logging

# Before anything else fix the bad ANSI blue
splatlog.setup(
    theme=splatlog.rich.override_ansi_colors(
        blue="#509dea", bright_blue="#439af4"
    )
)

LOG = splatlog.getLogger(__name__)

splatlog.setup(level="info", console="stdout")

LOG.info("Hey!")

INFO      __main__                                                                                                 
msg       Hey!

The default output is `STDOUT`.

In [ ]:
h = splatlog.to_console_handler(True)
h.console.file

In [14]:
import logging


splatlog.setup(level="info", console="debug")

# Does _not_ show
LOG.debug("Shown?")

ch = splatlog.get_named_handler("console")
assert isinstance(ch, splatlog.RichHandler)
assert ch.level == splatlog.DEBUG
LOG.info(
    "levels",
    root_handler=logging.getLevelName(logging.getLogger().level),
    console_handler=ch.get_level_name(),
)

INFO      __main__                                                                                                 
msg       levels                                                                                                   
data      root_handler    str          INFO                                                                        
          console_handler str          DEBUG

In [15]:
splatlog.setup(level=splatlog.NOTSET, console="info")

LOG.info("shown?")
LOG.debug("shown?")

INFO      __main__                                                                                                 
msg       shown?

In [16]:
logging.getLevelNamesMapping()

{'CRITICAL': 50,
 'FATAL': 50,
 'ERROR': 40,
 'WARN': 30,
 'WARNING': 30,
 'INFO': 20,
 'DEBUG': 10,
 'NOTSET': 0}

In [17]:
logging.getLevelName("hey")

'Level hey'

Custom Named Handler
------------------------------------------------------------------------------

In [18]:
@splatlog.named_handler("custom", on_conflict="replace")
def cast_custom_handler(value: Any) -> logging.Handler | None:
    LOG.info("casting custom handler...", value=value)

    #
    if value is None or value is False:
        return None

    if isinstance(value, Mapping):
        handler = rich.logging.RichHandler(**value)
        LOG.info("Constructed from mapping", handler=handler)
        return handler

    if splatlog.is_level(value):
        handler = rich.logging.RichHandler(level=value)
        LOG.info(
            "Constructed from level",
            handler=handler,
            handler_level=logging.getLevelName(handler.level),
        )
        return handler

    if isinstance(value, Console):
        handler = rich.logging.RichHandler(console=value)
        LOG.info(f"Constructed from {Console!r}", handler=handler)
        return handler

    raise ValueError(f"unexpected value: {value!r}")


splatlog.setup(custom=False)

LOG.info("custom?")


INFO      __main__.cast_custom_handler                                                                             
msg       casting custom handler...                                                                                
data      value       bool          False

INFO      __main__                                                                                                 
msg       custom?

In [19]:
from typing import IO, BinaryIO, TextIO
import os
import sys
import ipykernel
from splatlog.lib import satisfies

__file__ = ipykernel.get_connection_file()

for fio in [
    sys.stdout,
    sys.stderr,
    open(__file__, "r"),
    open(__file__, "r", encoding="utf-8"),
    open(__file__, "w", encoding="utf-8"),
    open(__file__, "wb"),
]:
    LOG.info(
        "satisfies?",
        fio=fio,
        sat_io_str=satisfies(fio, IO[str]),
        sat_binary_io=satisfies(fio, BinaryIO),
        sat_text_io=satisfies(fio, TextIO),
    )

INFO      __main__                                                                                                 
msg       satisfies?                                                                                               
data      fio           ipykernel                    <ipykernel.iostream.OutStream object at 0x104900730>          
                          .iostream                                                                                
                            .OutStream                                                                             
          sat_io_str    bool                         True                                                          
          sat_binary_io bool                         False                                                         
          sat_text_io   bool                         True

INFO      __main__                                                                                                 
msg       satisfies?                                                                                               
data      fio           ipykernel                    <ipykernel.iostream.OutStream object at 0x104900a60>          
                          .iostream                                                                                
                            .OutStream                                                                             
          sat_io_str    bool                         True                                                          
          sat_binary_io bool                         False                                                         
          sat_text_io   bool                         True

INFO      __main__                                                                                                 
msg       satisfies?                                                                                               
data      fio           _io               <_io.TextIOWrapper                                                       
                          .TextIOWrapper  name='/Users/nrser/Library/Jupyter/runtime/kernel-v36f53fbf2bc5a0ce0e994…
                                          mode='r' encoding='UTF-8'>                                               
          sat_io_str    bool              True                                                                     
          sat_binary_io bool              False                                                                    
          sat_text_io   bool              True

INFO      __main__                                                                                                 
msg       satisfies?                                                                                               
data      fio           _io               <_io.TextIOWrapper                                                       
                          .TextIOWrapper  name='/Users/nrser/Library/Jupyter/runtime/kernel-v36f53fbf2bc5a0ce0e994…
                                          mode='r' encoding='utf-8'>                                               
          sat_io_str    bool              True                                                                     
          sat_binary_io bool              False                                                                    
          sat_text_io   bool              True

INFO      __main__                                                                                                 
msg       satisfies?                                                                                               
data      fio           _io               <_io.TextIOWrapper                                                       
                          .TextIOWrapper  name='/Users/nrser/Library/Jupyter/runtime/kernel-v36f53fbf2bc5a0ce0e994…
                                          mode='w' encoding='utf-8'>                                               
          sat_io_str    bool              True                                                                     
          sat_binary_io bool              False                                                                    
          sat_text_io   bool              True

INFO      __main__                                                                                                 
msg       satisfies?                                                                                               
data      fio           _io                <_io.BufferedWriter                                                     
                          .BufferedWriter  name='/Users/nrser/Library/Jupyter/runtime/kernel-v36f53fbf2bc5a0ce0e99…
          sat_io_str    bool               False                                                                   
          sat_binary_io bool               True                                                                    
          sat_text_io   bool               False

In [20]:
c_0 = splatlog.rich.to_console(None)
LOG.info("default console", file=c_0.file)

cap = splatlog.rich.capture_riches(
    splatlog.rich.ntv_table({"x": 1, "y": 2}),
    console=c_0,
)

cap

INFO      __main__                                                                                                 
msg       default console                                                                                          
data      file        ipykernel                    <ipykernel.iostream.OutStream object at 0x104900a60>            
                        .iostream                                                                                  
                          .OutStream

'\x1bx          \x1b\x1b \x1b\x1bint         \x1b \x1b1\x1b          \n\x1by          \x1b\x1b \x1b\x1bint         \x1b \x1b2\x1b          \n'

Export (JSON) Handler
------------------------------------------------------------------------------

Play-testing the `export` named handler.

In [21]:
from io import StringIO
import splatlog
import json

sio = StringIO()
splatlog.setup(level="debug", console=True, export=sio)


def get_export():
    out_json = sio.getvalue()
    out = []
    for line in out_json.splitlines():
        out.append(json.loads(line))
    sio.seek(0)
    sio.truncate()
    return out


LOG = splatlog.getLogger(__name__)
LOG.info("hey", ho="let's go")
out = get_export()
out

INFO      __main__                                                                                                 
msg       hey                                                                                                      
data      ho          str          let's go

[{'t': '2025-12-08T15:04:18.088696-08:00',
  'level': 'INFO',
  'name': '__main__',
  'file': '/private/tmp/ipykernel_36009/205227518.py',
  'line': 20,
  'msg': 'hey',
  'data': {'ho': "let's go"}}]

### Rich Messages ###

I bumped into this feature when figuring out if I could get rid of the
`splatlog.lib.rich.theme.DEFAULT_CONSOLE` (we can) because it doesn't mesh well
with setting a default theme (something I'm actually using), and the only place
the default theme was being used was in `splatlog.lib.rich.capture_riches`,
which is used to test if `is_rich(logging.LogRecord.msg)`, and if so capture 
the rich rendering output to return.

This was used for something, though I cna't for the life of me think what..? 
The following example is about as contrived as possible, using a 
Name/Type/Value (NVT) table as the log message.

In [22]:
LOG.info(splatlog.rich.ntv_table({"x": 1, "y": 2}))
get_export()

INFO      __main__                                                                                                 
msg       x           int          1                                                                               
          y           int          2

[{'t': '2025-12-08T15:04:18.092831-08:00',
  'level': 'INFO',
  'name': '__main__',
  'file': '/private/tmp/ipykernel_36009/3764767004.py',
  'line': 1,
  'msg': '\x1bx          \x1b\x1b \x1b\x1bint         \x1b \x1b1\x1b          \n\x1by          \x1b\x1b \x1b\x1bint         \x1b \x1b2\x1b          \n'}]

The ANSI string encoding of that table is a terrible mess... but also it's
ridiculous to stick a table in the message. I'm trying to imagine what this
approach was actually _for_... I've tried searching through a project that uses
`splatlog` but haven't found any examples.

I can only guess that the use case was something dumb like using [Rich Console Markup](https://rich.readthedocs.io/en/latest/markup.html) in log messages.

In [23]:
from rich.text import Text

rich_msg = Text.from_markup("[bold red]alert![/bold red] Something happened")

LOG.info(rich_msg)
get_export()

INFO      __main__                                                                                                 
msg       alert! Something happened

[{'t': '2025-12-08T15:04:18.097762-08:00',
  'level': 'INFO',
  'name': '__main__',
  'file': '/private/tmp/ipykernel_36009/4095036149.py',
  'line': 5,
  'msg': '\x1balert!\x1b Something happened\n'}]

In [29]:
LOG.info("[bold red]alert![/bold red] Something happened")
get_export()

INFO      __main__                                                                                                 
msg       alert! Something happened

[{'t': '2025-12-08T15:05:11.681884-08:00',
  'level': 'INFO',
  'name': '__main__',
  'file': '/private/tmp/ipykernel_36009/735551129.py',
  'line': 1,
  'msg': 'alert! Something happened'}]

Ok, that's not as terrible, but I'm still questionable about using the ANSI coded string.

Rich has other options, like exporting HTML:

In [25]:
from io import StringIO
from rich.console import Console
from rich.text import Text
from IPython.display import HTML, display
import re

rich_msg = Text.from_markup("[bold red]alert![/bold red] Something happened")

f_rec = StringIO()
c_rec = Console(
    # Need this, as otherwise the Jupyter detection will result in `file=` not
    # working (WTF..?)
    force_jupyter=False,
    # Where the console should write to
    file=f_rec,
    # Force terminal control codes
    force_terminal=True,
    # Boolean to enable recording of terminal output
    record=True,
)

# Why does this print to output?
c_rec.print(rich_msg)
html = c_rec.export_html(inline_styles=True)

m = re.search(r"(?is)<body[^>]*>(.*?)</body\s*>", html)
body = m.group(1) if m else ""
print(body)
display(HTML(body))


    <pre style="font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace"><code style="font-family:inherit"><span style="color: #800000; text-decoration-color: #800000; font-weight: bold">alert!</span> Something happened
</code></pre>



In [26]:
c_rec.print("Hey [bold]now[/bold]!")
print(c_rec.export_svg())

<svg class="rich-terminal" viewBox="0 0 994 74.4" xmlns="http://www.w3.org/2000/svg">
    <!-- Generated with Rich https://www.textualize.io -->
    <style>

    @font-face {
        font-family: "Fira Code";
        src: local("FiraCode-Regular"),
                url("https://cdnjs.cloudflare.com/ajax/libs/firacode/6.2.0/woff2/FiraCode-Regular.woff2") format("woff2"),
                url("https://cdnjs.cloudflare.com/ajax/libs/firacode/6.2.0/woff/FiraCode-Regular.woff") format("woff");
        font-style: normal;
        font-weight: 400;
    }
    @font-face {
        font-family: "Fira Code";
        src: local("FiraCode-Bold"),
                url("https://cdnjs.cloudflare.com/ajax/libs/firacode/6.2.0/woff2/FiraCode-Bold.woff2") format("woff2"),
                url("https://cdnjs.cloudflare.com/ajax/libs/firacode/6.2.0/woff/FiraCode-Bold.woff") format("woff");
        font-style: bold;
        font-weight: 700;
    }

    .terminal-2875727452-matrix {
        font-family: Fira Code

In [27]:
import json

json.dumps(rich_msg)

TypeError: Object of type Text is not JSON serializable

In [30]:
splatlog.setup(blah=123)

KeyError: 'blah'